In [250]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from tensorflow.keras import backend as K




In [142]:
ls ../data

country_code_lookup.csv  yemen_batch0.csv  yemen_batch0.pkl  yemen_batch1.pkl
event_code_lookup.csv    yemen_batch0.pk   yemen_batch1.csv


In [221]:
yemen_df0 = pd.read_csv('../data/yemen_batch0.csv', index_col=0)
yemen_df0.head(2)

,Actor1Name,Actor2Name,QuadClass,GoldsteinScale,NumArticles,AvgTone,IsHumanitarianCrisis,sum,count,mean
SQLDATE,,,,,,,,,,
2014-02-19,ADEN,BRITAIN,3,-6.5,3,-2.103250,False,0,9,0.0
2014-02-19,YEMENI,LIBYA,1,1.9,2,-6.102747,False,0,9,0.0


In [180]:
def load_data(filename, idx):
    df = pd.read_csv('../data/{}{}.csv'.format(filename, idx), index_col=0)
    features = ['Actor1Name','Actor2Name','QuadClass','GoldsteinScale','NumArticles','AvgTone']
    categorical_features = ['Actor1Name','Actor2Name']
    column_trans = make_column_transformer(
                        (OneHotEncoder(handle_unknown = 'ignore'),  categorical_features),
                        remainder='passthrough')
    df.fillna('None', inplace=True)
    X_train = df[features]
    y_train = df['mean']
    return X_train, y_train

In [195]:
def batch_generator(filename,n):
    idx = 0
    while idx<n:
        yield load_data(filename=filename, idx=idx)
        idx+=1

In [228]:
filename = 'yemen_batch'
gen = batch_generator(filename, 2)

In [229]:
# next(gen)

In [232]:
features_ohe = yemen_df0.Actor1Name.value_counts()[yemen_df0.Actor1Name.value_counts().values > 50].keys()

In [241]:
yemen_df0_sub = yemen_df0[yemen_df0.Actor1Name.isin(features_ohe)]
yemen_df0_sub.head()

,Actor1Name,Actor2Name,QuadClass,GoldsteinScale,NumArticles,AvgTone,IsHumanitarianCrisis,sum,count,mean
SQLDATE,,,,,,,,,,
2014-02-19,ADEN,BRITAIN,3,-6.5,3,-2.103250,False,0,9,0.0
2014-02-19,YEMENI,LIBYA,1,1.9,2,-6.102747,False,0,9,0.0
2014-02-19,LIBYA,YEMENI,1,2.8,2,-6.102747,False,0,9,0.0
2014-02-19,YEMENI,LIBYA,1,1.9,1,-6.243603,False,0,9,0.0
2014-02-19,LIBYA,YEMENI,1,2.8,1,-6.243603,False,0,9,0.0


In [245]:
yemen_df0_sub.fillna('None', inplace=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [259]:
col = ['Actor1Name', 'Actor2Name', 'IsHumanitarianCrisis']
column_trans = make_column_transformer(
                        (OneHotEncoder(handle_unknown = 'ignore'),  col),
                        remainder='passthrough')
column_trans.fit_transform(yemen_df0_sub).toarray()

array([[0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        9.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        9.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        9.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.2000000e+01,
        3.9900000e+02, 3.0075188e-02],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 1.2000000e+01,
        3.9900000e+02, 3.0075188e-02],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.2000000e+01,
        3.9900000e+02, 3.0075188e-02]])

In [254]:
onehotencodeer = OneHotEncoder(handle_unknown='ignore')
onehotencodeer.fit_transform(yemen_df0_sub)

<12274x5255 sparse matrix of type '<class 'numpy.float64'>'
	with 122740 stored elements in Compressed Sparse Row format>

In [262]:
len(yemen_df0['mean'])

18435

In [236]:
# yemen_df0.Actor2Name.value_counts()[yemen_df0.Actor2Name.value_counts().values > 50].keys()

In [202]:
model = Sequential()
model.add(Dense(units=1, input_dim=6))
model.compile(loss='mean_squared_error', optimizer='sgd')

In [203]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 7         
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________


In [206]:
features = ['Actor1Name','Actor2Name','QuadClass','GoldsteinScale','NumArticles','AvgTone']
X_train = yemen_df0[features]
y_train = yemen_df0['mean']

In [212]:
# model.fit(X_train, y_train)

# Appendix

In [73]:
def sample_generator():
    i = 0
    while True:
        i += 1
        yield(i)
        

In [82]:
gen = sample_generator()

In [84]:
next(gen)

2

In [ ]:
def load_data(train_df,idx,
              batch_size):
    df = pd.read_csv(
                  Train_df, skiprows=idx*batch_size,
                  nrows=batch_size)
    x = df.iloc[:,1:]
         
    y = df.iloc[:,0]
    return (np.array(x), np_utils.to_categorical(y))

In [91]:
def load_data(filename, idx):
    df = pd.read_csv('../data/{}{}.csv'.format(filename, idx), index_col=0)
    return df.head(1)

idx = '0'
filename = 'yemen_batch'
load_data(filename, idx)

,Actor1Name,Actor2Name,QuadClass,GoldsteinScale,NumArticles,AvgTone,IsHumanitarianCrisis,sum,count,mean
SQLDATE,,,,,,,,,,
2014-02-19,ADEN,BRITAIN,3,-6.5,3,-2.10325,False,0,9,0.0


In [92]:
def sample_generator(filename):
    idx = 0
    while True:
        yield(load_data(filename, idx))
    

In [93]:
gen = sample_generator(filename)

In [94]:
gen

<generator object sample_generator at 0x7faf0e267bf8>

In [95]:
for x in range(5):
    print(x)
    

0
1
2
3
4


In [135]:
def sam_generator(limit):
    i = 0
    while i<limit:
        i += 1
        yield i




In [136]:
gen = sam_generator(4)

In [141]:
next(gen)

StopIteration: 